# Cell Magics in IPython

IPython has a system of commands we call 'magics' that provide a mini command language that is orthogonal to the syntax of Python and is extensible by the user with new commands.  Magics are meant to be typed interactively, so they use command-line conventions, such as using whitespace for separating arguments, dashes for options and other conventions typical of a command-line environment.

Magics come in two kinds:

* Line magics: these are commands prepended by one `%` character and whose arguments only extend to the end of the current line.
* Cell magics: these use *two* percent characters as a marker (`%%`), and they receive as argument *both* the current line where they are declared and the whole body of the cell.  Note that cell magics can *only* be used as the first line in a cell, and as a general principle they can't be 'stacked' (i.e. you can only use one cell magic per cell).  A few of them, because of how they operate, can be stacked, but that is something you will discover on a case by case basis.

The `%lsmagic` magic is used to list all available magics, and it will show both line and cell magics currently defined:

In [1]:
%lsmagic

Available line magics:
%alias  %alias_magic  %autocall  %automagic  %autosave  %bookmark  %cat  %cd  %clear  %colors  %config  %connect_info  %cp  %debug  %dhist  %dirs  %doctest_mode  %ed  %edit  %env  %gui  %hist  %history  %install_default_config  %install_ext  %install_profiles  %killbgscripts  %ldir  %less  %lf  %lk  %ll  %load  %load_ext  %loadpy  %logoff  %logon  %logstart  %logstate  %logstop  %ls  %lsmagic  %lx  %macro  %magic  %man  %matplotlib  %mkdir  %more  %mv  %notebook  %page  %pastebin  %pdb  %pdef  %pdoc  %pfile  %pinfo  %pinfo2  %popd  %pprint  %precision  %profile  %prun  %psearch  %psource  %pushd  %pwd  %pycat  %pylab  %qtconsole  %quickref  %recall  %rehashx  %reload_ext  %rep  %rerun  %reset  %reset_selective  %rm  %rmdir  %run  %save  %sc  %set_env  %store  %sx  %system  %tb  %time  %timeit  %unalias  %unload_ext  %who  %who_ls  %whos  %xdel  %xmode

Available cell magics:
%%!  %%HTML  %%SVG  %%bash  %%capture  %%debug  %%file  %%html  %%javascript  %%latex  %%

Since in the introductory section we already covered the most frequently used line magics, we will focus here on the cell magics, which offer a great amount of power.

Let's load matplotlib and numpy so we can use numerics/plotting at will later on.

In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

## <!--====--> Some simple cell magics

Timing the execution of code; the 'timeit' magic exists both in line and cell form:

In [3]:
%timeit np.linalg.eigvals(np.random.rand(100,100))

100 loops, best of 3: 7.06 ms per loop


In [4]:
%%timeit a = np.random.rand(100, 100)
np.linalg.eigvals(a)

100 loops, best of 3: 7.4 ms per loop


The `%%capture` magic can be used to capture the stdout/err of any block of python code, either to discard it (if it's noise to you) or to store it in a variable for later use:

In [5]:
%%capture capt
from __future__ import print_function
import sys
print('Hello stdout')
print('and stderr', file=sys.stderr)

In [6]:
capt.stdout, capt.stderr

('Hello stdout\n', 'and stderr\n')

In [7]:
capt.show()

Hello stdout


and stderr


The `%%writefile` magic is a very useful tool that writes the cell contents as a named file:

In [8]:
%%writefile foo.py
print('Hello world')

Writing foo.py


In [9]:
%run foo

Hello world


## <!--====--> Magics for running code under other interpreters

IPython has a `%%script` cell magic, which lets you run a cell in
a subprocess of any interpreter on your system, such as: bash, ruby, perl, zsh, R, etc.

It can even be a script of your own, which expects input on stdin.

To use it, simply pass a path or shell command to the program you want to run on the `%%script` line,
and the rest of the cell will be run by that script, and stdout/err from the subprocess are captured and displayed.

In [10]:
%%script python2
import sys
print 'hello from Python %s' % sys.version

hello from Python 2.7.9 (default, Jan 29 2015, 06:27:40) 
[GCC 4.2.1 Compatible Apple LLVM 6.0 (clang-600.0.56)]


In [11]:
%%script python3
import sys
print('hello from Python: %s' % sys.version)

hello from Python: 3.4.2 |Continuum Analytics, Inc.| (default, Oct 21 2014, 17:42:20) 
[GCC 4.2.1 (Apple Inc. build 5577)]


IPython also creates aliases for a few common interpreters, such as bash, ruby, perl, etc.

These are all equivalent to `%%script <name>`

In [12]:
%%ruby
puts "Hello from Ruby #{RUBY_VERSION}"

Hello from Ruby 2.0.0


In [13]:
%%bash
echo "hello from $BASH"

hello from /usr/local/bin/bash


## Capturing output

You can also capture stdout/err from these subprocesses into Python variables, instead of letting them go directly to stdout/err

In [14]:
%%bash
echo "hi, stdout"
echo "hello, stderr" >&2


hi, stdout


hello, stderr


In [15]:
%%bash --out output --err error
echo "hi, stdout"
echo "hello, stderr" >&2

In [16]:
print(error)
print(output)

hello, stderr

hi, stdout



## Background Scripts

These scripts can be run in the background, by adding the `--bg` flag.

When you do this, output is discarded unless you use the `--out/err`
flags to store output as above.

In [17]:
%%ruby --bg --out ruby_lines
for n in 1...10
    sleep 1
    puts "line #{n}"
    STDOUT.flush
end

Starting job # 0 in a separate thread.


When you do store output of a background thread, these are the stdout/err *pipes*,
rather than the text of the output.

In [18]:
ruby_lines

<_io.BufferedReader name=52>

In [19]:
print(ruby_lines.read().decode('utf8'))

line 1
line 2
line 3
line 4
line 5
line 6
line 7
line 8
line 9

